# What are we going to try to cover today?

# Context Managers and Decorators 

Let's import everything we need first.. 

In [52]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from contextlib import contextmanager
from datetime import datetime
from sklearn.metrics import classification_report 
import json
from functools import wraps 
import numpy as np


## What are context managers???

Context managers allow you to allocate and release resources precisely when you want to.  - Taken from pythontips.com

Examples include operations where you might want to carry out an action and then yield the workspace to a different process before completing the action. 



In [27]:
@contextmanager
def conversation_printer():
    print("Hello! What is your name?!")
    yield
    print("Nice to meet you. Welcome to an advance python crash course!")


with conversation_printer():
    print("My name is Daria")

Hello! What is your name?!
My name is Daria
Nice to meet you. Welcome to an advance python crash course!


What are some of the other more common ways we use context managers in Python?
File writing! Have you ever seen this?


In [28]:
with open("dummy_file.txt", "w+") as f:
    f.write("Hello!")

What does this look like under the hood?

In [32]:
@contextmanager
def open(name, mode):
    f = open(name, mode)
    try:
        yield f
    finally:
        f.close()

Quick distinction here is that you can acocunt for fault tolerance within a context manager by
using try/except/finally. With this you can ensure that if the yielded process is of not acceptable kind
the operation does not complete or completes in a specific logic.


## Decorators wrap functions and can be used to provide extra functionality, or decorating processes

In [34]:
def cat():
    return "Cat"
def dog():
    return "Dog"
def piglet():
    return "piglet"

def sound_maker(func):
    def wrapper():
        animal = func()
        if animal =="Cat":
            print('Meow')
        elif animal =="Dog":
            print("Woof")
        else:
            print("Not sure what this animal is?!")
    return wrapper

wrapped_object = sound_maker(cat)
wrapped_object()

Meow


Let's take this a step further... using the @ magic symbol while not having to utilize sound_maker() at all!


In [35]:
def sound_maker(func):
    def wrapper():
        func()
        print("Says Meow")
    return wrapper

@sound_maker
def animal():
    print("Cat")

animal()

Cat
Says Meow


## How do we use context managers and decorators to take our machine learning to the next level?

In [4]:
from sklearn import datasets
import pandas as pd
iris = datasets.load_iris(as_frame = True)

y = iris['target']
x = iris['data']

In [5]:
x.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

Train test split the data and load necessary classes/functions

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y)

Fit models

### Take 1 .... 

In [6]:
rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
accuracy_score(ytest, y_pred)

0.9210526315789473

### Take 2 ... 

In [7]:
rf = AdaBoostClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
accuracy_score(ytest, y_pred)

0.9473684210526315

### Take 3 ... 

In [8]:
rf = ExtraTreesClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
accuracy_score(ytest, y_pred)

0.9210526315789473

### And so on ... Isn't this annoying ? Also not DRY (Don't Repeat Yourself) 

Make a function for it... 

In [8]:
def fitter(model, xtrain, xtest, ytrain, ytest): 
    md = model()
    md.fit(xtrain, ytrain)
    y_pred = md.predict(xtest)
    return accuracy_score(ytest, y_pred), y_pred, model

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y)
score, prediction, fmodel = fitter(RandomForestClassifier, xtrain, xtest, ytrain, ytest)

In [10]:
print(score)

1.0


What else is better? Let's keep track of what we run with that function and its outputs 

In [14]:
registry = {}
def model_registry(function): 
    @wraps(function)
    def wrapped(*args, **kwargs): 
        score, prediction, fmodel = function(*args, **kwargs)
        registry[args[0].__name__] = {'score': score, 'prediction': prediction} 
        return score, prediction, fmodel
    return wrapped

In [15]:
@model_registry
def fitter(model, xtrain, xtest, ytrain, ytest): 
    md = model()
    md.fit(xtrain, ytrain)
    y_pred = md.predict(xtest)
    return accuracy_score(ytest, y_pred), y_pred, md

In [16]:
score, prediction, fmodel = fitter(RandomForestClassifier, xtrain, xtest, ytrain, ytest)
score, prediction, fmodel = fitter(ExtraTreesClassifier, xtrain, xtest, ytrain, ytest)
score, prediction, fmodel = fitter(AdaBoostClassifier, xtrain, xtest, ytrain, ytest)

In [17]:
registry

{'RandomForestClassifier': {'score': 0.9210526315789473,
  'prediction': array([1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0,
         2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0])},
 'ExtraTreesClassifier': {'score': 0.9210526315789473,
  'prediction': array([1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0,
         2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0])},
 'AdaBoostClassifier': {'score': 0.9210526315789473,
  'prediction': array([1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 1, 0, 2, 0, 1, 0, 0, 0, 0,
         2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0])}}

## Context Managers

In [19]:
@contextmanager
def write_signature(*args, **kwargs): 
    with open(*args, **kwargs) as f: 
        yield f 

In [20]:
def model_registry(function): 
    @wraps(function)
    def wrapped(*args, **kwargs): 
        score, prediction, fmodel = function(*args, **kwargs)
        registry[args[0].__name__] = {'score': score, 'prediction': prediction} 
        with write_signature("{}_report.txt".format(datetime.today().strftime('%Y-%m-%d')), mode = 'w+') as f: 
            report = classification_report(args[4], prediction)
            f.write(args[0].__name__ + "\n" + report)
        return score, prediction, fmodel
    return wrapped

In [21]:
@model_registry
def fitter(model, xtrain, xtest, ytrain, ytest): 
    md = model()
    md.fit(xtrain, ytrain)
    y_pred = md.predict(xtest)
    return accuracy_score(ytest, y_pred), y_pred, md

In [22]:
registry = {}
score, prediction, fmodel = fitter(AdaBoostClassifier, xtrain, xtest, ytrain, ytest)

In [23]:
registry

{'AdaBoostClassifier': {'score': 0.9736842105263158,
  'prediction': array([2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 2, 1, 0,
         1, 1, 0, 2, 0, 0, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2])}}

# Python Classes, Methods and Inheritance

In [45]:
class MyFancyClass:
    def __init__(self): 
        self.my_attribute = "I am fancy!"
        pass 
    def my_fancy_method(self):
        print("I am busy, dont' bother me. I am fancy!")
#     def __call__(self):
#         pass
#     def __str__(self): 
#         pass 
#     def __repr__(self): 
#         pass
#     def __get__(self): 
#         pass
#     def __set__(self): 
#         pass 
    # and more!!! 
    

You can think of classess as cookiecutters for or factories for producing similar products with variations. For example you can have a class for creating a car, but cars can be of different brands: BMW, Honda, Ferrari and ect! 

Methods are functions that are inherent to classes. Attributes are variables that are assigned to a class. Class attributes are same accross all classes, object attributes are object speicfic. We won't worry about class methods or attributes in this section but we'll touch on them in later sections.

In [46]:
my_class = MyFancyClass()

In [47]:
my_class.my_attribute

'I am fancy!'

In [48]:
my_class.my_fancy_method()

I am busy, dont' bother me. I am fancy!


In [50]:
# Examples from "Learn Python 3 the Hard Way" by Zed A Shaw

class Parent(object):
    def __init__(self):
        self.phone_type = "iphone"
        # self.mood = "happy" # if I change this attribute here, it won't get inherited!

    def implicit(self):
        print(f'I am a person with {self.phone_type}')

    def override(self):
        print("I have an {}".format(self.phone_type))

    def altered(self):
        self.mood = "happy"
        print("Time to hug child!")


class Child(Parent):  # Child will inherit from Parent
    def __init__(self):
        self.phone_type = "samsung"
        self.mood = "sad"

    def override(self):
        print("Override in Child")
        print("I have an {}".format(self.phone_type))

    def altered(self):
        print("My current mood is {}".format(self.mood))
        super().altered()
        print("And now my mood is {}".format(self.mood))


parent = Parent()
child = Child()

# parent.implicit()
# child.implicit()  # we'll inherit directly from the Parent class
#
# parent.override()
# child.override()  # we override the parent class. We want the functionality of this class to behave differently
#
child.altered()  # note that when I call super I can change the attributes in the child class!


My current mood is sad
Time to hug child!
And now my mood is happy


In [49]:
# From Learn Web Development with Python by Fabrizio Romano

# Using inheritance often leads to multiple inheritance problems, known as "diamond inheritance"


class A:
    def print_name(self):
      print("a")


class B(A):
    def print_name(self):
      print("b")


class C(A):
    def print_name(self):
      print("c")


class D(B, C):
    pass


d = D()
d.print_name()  # label could almost be either c or b!
print(d.__class__.__mro__)  # MRO = method resolution order, a way to see how to go up the ladder of inheritance


b
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


## Why would you need classes? 

Have you ever wondered, how can create my own scikit-learn like model? You can do precisely that with classes. Moreover you can take scikit-learn classes and modify them however you like!

In [53]:
class AdjustmentTransformer(object): 
    def __init__(self, input_array):
        self.array = input_array
    def fit(self):
        mean = np.mean(self.array)
        self.adjusted_array = self.array + mean 
    def predict(self): 
        return self.adjusted_array

In [54]:
transformer = AdjustmentTransformer(np.array([37.5, 22, 54.2, 11.75, 90.89]))

In [55]:
transformer.fit()

In [56]:
transformer.predict()

array([ 80.768,  65.268,  97.468,  55.018, 134.158])